# Modelos del lenguaje con RNNs

En esta parte, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar esta actividad con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/d/d8/El_ingenioso_hidalgo_don_Quijote_de_la_Mancha.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano antiguo de El Ingenioso Hidalgo Don Quijote de la Mancha, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro de Don Quijote, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [ ]:
import numpy as np
import keras
import random
import matplotlib.pyplot as plt
import re
import unicodedata

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


from collections import Counter

In [ ]:
path = keras.utils.get_file(
    fname="don_quijote.txt",
    origin=" https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/textos/Don_Quijote_de_la_Mancha.txt"
)

2151176/2151176 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [ ]:
# Abrir y leer el contenido
with open(path, 'r', encoding='utf-8') as f:
    texto = f.read()

# Mostrar los primeros 500 caracteres (por ejemplo)
print(texto[:500])

Capítulo primero. Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha


En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. Una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda. El resto della co


Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado, con el comienzo tan característico del Quijote.

In [ ]:
print("Longitud del texto: {}".format(len(texto)))
print(texto[0:300])

Longitud del texto: 2071198
Capítulo primero. Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha


En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. Una olla de algo más


## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*El ingenioso don Qui*" -> predicción: **j**
* "*El ingenioso don Quij*" -> predicción: **o**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:   El ingenioso don Quijot
>* *Output*: l ingenioso don Quijote

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:   El ingenioso don Quijot
>* *Output*: e

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante.

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de caracteres con el siguiente carácter a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* caracteres (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de los caracteres y mapas de caracteres

Antes que nada, necesitamos saber qué caracteres aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_chars* - 1 en el modelo. Obtener:


1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_chars* - 1. Por ejemplo, {'a': 0, 'b': 1, ...}
3.   Diccionario reverso de índices a caracteres: {0: 'a', 1: 'b', ...}


In [ ]:
def limpiar_texto(texto):
    texto = texto.lower()

    # Eliminar acentos (pero preservar la ñ manualmente)
    texto = texto.replace('ñ', '__enie__')  # marcador temporal
    texto_normalizado = unicodedata.normalize('NFD', texto)
    texto_sin_acentos = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
    texto_sin_acentos = texto_sin_acentos.replace('__enie__', 'ñ')  # restaurar ñ

    # Eliminar caracteres especiales, dejar solo letras, números, espacios y ñ
    texto_limpio = re.sub(r'[^a-z0-9ñ ]+', '', texto_sin_acentos)

    return texto_limpio


resultado = limpiar_texto(texto)


# Obtener el conjunto de caracteres únicos
chars = sorted(list(set(resultado)))

# Número de caracteres únicos
num_chars = len(chars)
print(f"Número de caracteres únicos: {num_chars}")

# Crear diccionario de caracteres a índices
char_to_index = {char: index for index, char in enumerate(chars)}

# Crear diccionario de índices a caracteres
index_to_char = {index: char for index, char in enumerate(chars)}

# Imprimir algunos ejemplos para verificar
print("\nEjemplo de char_to_index:")
for i in range(len(char_to_index)):
    print(f"'{chars[i]}': {char_to_index[chars[i]]}")

print("\nEjemplo de index_to_char:")
for i in range(len(char_to_index)):
    print(f"{i}: '{index_to_char[i]}'")


Número de caracteres únicos: 35

Ejemplo de char_to_index:
' ': 0
'0': 1
'1': 2
'2': 3
'3': 4
'4': 5
'5': 6
'6': 7
'7': 8
'a': 9
'b': 10
'c': 11
'd': 12
'e': 13
'f': 14
'g': 15
'h': 16
'i': 17
'j': 18
'l': 19
'm': 20
'n': 21
'o': 22
'p': 23
'q': 24
'r': 25
's': 26
't': 27
'u': 28
'v': 29
'w': 30
'x': 31
'y': 32
'z': 33
'ñ': 34

Ejemplo de index_to_char:
0: ' '
1: '0'
2: '1'
3: '2'
4: '3'
5: '4'
6: '5'
7: '6'
8: '7'
9: 'a'
10: 'b'
11: 'c'
12: 'd'
13: 'e'
14: 'f'
15: 'g'
16: 'h'
17: 'i'
18: 'j'
19: 'l'
20: 'm'
21: 'n'
22: 'o'
23: 'p'
24: 'q'
25: 'r'
26: 's'
27: 't'
28: 'u'
29: 'v'
30: 'w'
31: 'x'
32: 'y'
33: 'z'
34: 'ñ'


En este bloque se genera una función que se encarga de limpiar el texto para poder mentener unicamente los caracteres necesarios para completar el texto, se realizaron los siguientes cambios.

- Se utilizo normalizacion utilizando unicode en formato NFD, esto lo que hace es separar las tildes y caracteres especiales presentes en una letra, ejemplo: á → a ´.

- Teniendo en cuenta el punto anterior, la letra Ñ se mantuvo por lo que se transformo de manera temporal a __enie__ para evitar que el formato unicode NFD separe la virgulilla y transforme la "Ñ" a una "N ~".

- Se eliminaron todos los signos de puntuacion, pregunta, exclamacion, etc. Solo se mantuvieron las letras del abecedario, numeros, espacios en blanco y la letra "Ñ" que si bien puede tomarse como un carater especial, es considerada una letra.

Finalmente cuando se realiza la limpieza se generan los caracteres de indice a caracteres y caracter a indice para podere valuar como quedarian los datos, con un valor numerico entero para poder realizar el entrenamiento.

#### 2.2. Obtención de secuencias de entrada y carácter a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y los correspondientes caracteres a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH caracteres y el siguiente caracter a predecir. Una vez hecho, desplazarse un carácter a la izquierda y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y los caracteres a predecir en una variable ***next_chars***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 5, tendríamos

* *sequences* = ["Don Q", "on Qu", "n Qui", " Quij", "Quijo", "uijot"]
* *next_chars* = ['u', 'i', 'j', 'o', 't', 'e']

In [ ]:
# Definimos el tamaño de las secuencias. Puedes dejar este valor por defecto.
SEQ_LENGTH = 60

sequences = []
next_chars = []

# Recorremos el texto y extraemos las secuencias de longitud fija y el siguiente carácter
for i in range(0, len(resultado) - SEQ_LENGTH):
    sequences.append(resultado[i: i + SEQ_LENGTH])     # secuencia de 30 caracteres
    next_chars.append(resultado[i + SEQ_LENGTH])        # siguiente carácter después de la secuencia

Manteniendo la explicacion principal, se generan secuencias con 60 caracteres en la variable secuences, que siempre sera una secuencia cortada donde falte 1 caracter, en la variable next_chars estara almacenado el caracter faltante de la secuencia. Esto es un enfoque tmado en clasificacion donde secuences es el valor de X y next_chars funciona como la Y (Valor de la clase o resultado a predecir).

Indicar el tamaño del training set que acabamos de generar.

In [ ]:
print(f"Número de secuencias: {len(sequences)}")
print(f"Número de caracteres objetivo: {len(next_chars)}")

Número de secuencias: 1968193
Número de caracteres objetivo: 1968193


Al generar las secuencias se generaron, a partir del texto, 1.968.193 secuencias diferentes, mismo numero de carateres o clases (Y).

In [ ]:
def es_secuencia_valida(seq):
    # Si es todo espacios o una sola letra repetida, no sirve
    return seq.strip() != "" and len(set(seq)) > 1

# Aplicar filtro
sequences_filtradas = []
next_chars_filtradas = []

for seq, next_char in zip(sequences, next_chars):
    if es_secuencia_valida(seq):
        sequences_filtradas.append(seq)
        next_chars_filtradas.append(next_char)

print(f"Secuencias útiles: {len(sequences_filtradas)} / {len(sequences)}")

Secuencias útiles: 1968193 / 1968193


Aqui se evalua la presencia de secuencias inutiles o que no entregan nada de relevancia al modelo como por ejemplo: Ahhhhh, que esto. En este caso no ocurre por el tamaño seteado de las secuencias (60 caracteres) lo cual impide secuencias asi, tal como se ve se mantuvieron todas las sencuencias y no se descarto ninguna.

Como el Quijote es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [ ]:
MAX_SEQUENCES = 350000

sequences = np.array(sequences_filtradas)
next_chars = np.array(next_chars_filtradas)

perm = np.random.permutation(len(sequences_filtradas))
sequences, next_chars = sequences[perm], next_chars[perm]
sequences, next_chars = np.array(sequences), np.array(next_chars)
sequences, next_chars = sequences[perm], next_chars[perm]
sequences, next_chars = list(sequences[:MAX_SEQUENCES]), list(next_chars[:MAX_SEQUENCES])

print(len(sequences))

350000


Por factores de consumo de recursos de Google colab, no se utilizaran el 1.968.193 datos, sino que se utilizara una fraccion de estos, especificamente 350.000, donde el codigo genera con la permutacion un bajareo de indices de las secuencias y sus siguientes carateres, esto para tomar los 350.000 datos de manera aleatoria.

#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar *one-hot encoding* para nuestros caracteres. Por ejemplo, si sólo tuviéramos 4 caracteres (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_chars)* e **y** tendrá shape *(num_sequences, num_chars)*.



In [ ]:
NUM_CHARS = 35  # Tu número de caracteres distintos aquí
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH), dtype=np.int32)
y = np.zeros((NUM_SEQUENCES,), dtype=np.int32)


for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t] = char_to_index[char]
    y[i] = char_to_index[next_chars[i]]  # siguiente caracter (1 por secuencia)

Este codigo genera nuestra separacion de datos en X e Y, utiliza el arreglo de caracteres a indices para guardar la secuencia en X (de manera numerica con el indice entero) y su siguiente carater en Y con el valor numerico ya listo para pasarse al modelo.

## 3. Definición del modelo y entrenamiento

Se tomo punto base el crear la LSTM con 128 unidades internas, utilizando softmax. Pero la arquitectura de la RNN se fue complejizando hasta dejar una arquitectura y preprocesamiento de datos mas completo buscando mejorar el rendimiento del modelo. Las tecnicas aplicadas como preprocesamiento y definicion del modelo son las siguientes:

- Generacion de pesos artificiales: Esto para poder sustentar un poco el desbalance de datos, cuya existencia en un set de datos no estructurado es casi inevitable.

- Aplicacion de Early Stopping: Debido al alto consumo de recursos para la RNN se aplico early stopping para evitar que el modelo siga consumiendo recursos en epocas que solo estan empeorando o manteniendo el mismo rendimiento.

- Uso de LSTM junto a bidirectional con 512 unidades de memoria: Para mejorar el rendimiento general del modelo se aplica una capa LSTM con bidireccional para que el modelo procese la secuencia de entrada de izquierda a derecha (como loa hace la LSTM tradicional) pero a su vez aplique un analisis de derecha a izquierda. Se apicaron 512 unidades de memoria, en contra dle punto inicial debido al bajo rendimiento que se entrego con esa prueba.

In [ ]:
conteo = Counter(next_chars)
total = sum(conteo.values())

# Frecuencia relativa de cada carácter
frecuencias = {char: count / total for char, count in conteo.items()}

# Peso inverso para cada clase
pesos = {char: 1.0 / freq for char, freq in frecuencias.items()}

max_peso = max(pesos.values())
pesos = {char: peso / max_peso for char, peso in pesos.items()}

sample_weights = np.array([pesos[c] for c in next_chars], dtype=np.float32)

Este bloque de codigo genera pesos artificiales calculando la frecuencia del siguiente caracter usando Counter, con esto se aplican pesos inversos, es decir, los caracteres mas frecuentes se les genera un peso menor a los menos frecuentes buscando compensar el desbalance de clases claramente presente.

In [ ]:
embedding_dim = 512
rnn_units = 512
early_stopping = EarlyStopping(monitor="loss", patience=5, min_delta=0.05, restore_best_weights=True)

model_2 = Sequential([
    Embedding(input_dim=NUM_CHARS, output_dim=embedding_dim),
    Bidirectional(LSTM(rnn_units, return_sequences=True, dropout=0.2)),
    Bidirectional(LSTM(rnn_units, return_sequences=False, dropout=0.2)),
    Dense(NUM_CHARS, activation='softmax')
])

model_2.compile(optimizer="adam", loss='sparse_categorical_crossentropy')
model_2.fit(X, y, batch_size=64, epochs=15, callbacks=[early_stopping], sample_weight=sample_weights)

Epoch 1/15
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 512s 92ms/step - loss: 2.8719e-04
Epoch 2/15
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 563s 93ms/step - loss: 2.1499e-04
Epoch 3/15
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 508s 93ms/step - loss: 2.3667e-04
Epoch 4/15
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 563s 93ms/step - loss: 2.1297e-04
Epoch 5/15
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 561s 93ms/step - loss: 1.8381e-04
Epoch 6/15
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 508s 93ms/step - loss: 1.7012e-04


Se aplico una RNN que se compone de las siguientes capas:

- Capa de embedding con dimesion de 512: Esto tomara los numeros enteros (los indices de los caracteres) y seran aplanados en un vector de 512 dimensiones. Esto puede mejorar bastante el rendimiento del modelo pero el consumo de recursos es mucho mayor

- 2 capas LSTM con bidirectional y 512 unidades de memoria (neuronas): Se utilizaron dos capas LSTM envoltas en capas Bidirectional, lo que permite que la red procese la información tanto en dirección hacia adelante como hacia atrás, capturando mejor el contexto en ambas direcciones. La primera capa devuelve una secuencia completa de características (una salida por cada paso de la secuencia), que luego es procesada por la segunda capa para extraer patrones más complejos de lenguaje. Esto mejora la capacidad del modelo para aprender dependencias a largo plazo en el texto.

- Capa densa de salida: La capa que nos entregara el siguiente caracter de la secuencia, como sucede en las redes de clasificacion tradicionales.

- Funcion de perdida: Se aplica Sparse_categorical_crossentropy debido a que no se esta utilizando un formato de one hot encoder, se utiliza esa funcion de perdida.


In [ ]:
def generate_multiple_texts(model, start_phrases, gen_length=100, temperature=1.0):
    for phrase in start_phrases:
        input_indices = [char_to_index[c] for c in phrase.lower()]
        input_seq = tf.expand_dims(input_indices, 0)
        generated = phrase

        for _ in range(gen_length):
            preds = model(input_seq)  # última predicción
            preds = preds / temperature
            next_id = tf.random.categorical(preds, num_samples=1).numpy()[0][0]
            next_char = index_to_char[next_id]

            generated += next_char
            input_indices.append(next_id)
            input_indices = input_indices[-SEQ_LENGTH:]
            input_seq = tf.expand_dims(input_indices, 0)

        print(f"\n🟡 Frase inicial: '{phrase}'")
        print(f"🔹 Predicción: {generated}")

Este código define una función llamada generate_multiple_texts que se encarga de generar texto automáticamente usando un modelo de red neuronal entrenado. Lo que hace, explicado en lenguaje natural, es lo siguiente:

La función toma una lista de frases iniciales (start_phrases) y, para cada una, empieza a predecir y generar texto carácter por carácter. Primero convierte la frase de entrada en una secuencia de números (índices de caracteres) que el modelo pueda entender. Luego, mientras no se alcance la cantidad deseada de caracteres (gen_length), la función alimenta esa secuencia al modelo, que devuelve una probabilidad para cada posible carácter siguiente.

Estas probabilidades se ajustan con un valor llamado temperature, que controla qué tan creativas o seguras son las predicciones (valores bajos = más conservadoras; valores altos = más creativas y variadas). Después, se escoge un carácter de forma aleatoria pero ponderada según esas probabilidades, se agrega a la secuencia y se repite el proceso, desplazando la ventana de entrada para siempre usar los últimos caracteres generados.

Al final, se imprime tanto la frase original como el texto generado automáticamente por el modelo, mostrando cómo continúa la red neuronal esa idea inicial.

In [ ]:
frases_prueba = [
    "don quijote ",
    "en un lugar de la ",
    "no ha mucho tiempo ",
    "vivia un hidalgo ",
    "de los de lanza en ",
    "era de complexion ",
    "el hidalgo salio al ",
    "andaba por los caminos ",
    "una mañana decidio "
]

generate_multiple_texts(model_2, frases_prueba, gen_length=60, temperature=0.5)


🟡 Frase inicial: 'don quijote '
🔹 Predicción: don quijote s1fgyavcyt5w1lmicgb7lgorcgxbl56qamfuoazummv6so0d1xjv0 vmoeñw

🟡 Frase inicial: 'en un lugar de la '
🔹 Predicción: en un lugar de la quubgy11ñuujzou7vñpbd25dañoqs6l00ozdn3gwz2x0wecggmjs2ynñev w

🟡 Frase inicial: 'no ha mucho tiempo '
🔹 Predicción: no ha mucho tiempo pjhltlclyba4u2d72icz45ife34rafpantdnaf0rnn27qltoq0mocg36jdod

🟡 Frase inicial: 'vivia un hidalgo '
🔹 Predicción: vivia un hidalgo en2y3fu4rhfjñyrj0eñwe svqu3xe7a7ayx607j1w0dsy v0c7 m4 31b45y

🟡 Frase inicial: 'de los de lanza en '
🔹 Predicción: de los de lanza en ytglbyalqlwnqb1rxpanjgisñr754grbotyzm 4 aqubn6wdiñ7ha6g4iryn

🟡 Frase inicial: 'era de complexion '
🔹 Predicción: era de complexion a1r7eyñaafllaf2cñf0ren zpgtzeoivhl4bodo12 yyvsxedtuie5ap06wn

🟡 Frase inicial: 'el hidalgo salio al '
🔹 Predicción: el hidalgo salio al 3lslquawbeoelq6rx4rczujhy n5a6hjbmnplxy7nrsteqogyyarb6wqc7j7

🟡 Frase inicial: 'andaba por los caminos '
🔹 Predicción: andaba por los caminos 7

Aqui se le entrega una lista de frases a la funcion para que el model orealice multiples predicciones, esto permite evaluar de forma general y evitar evaluar el modelo con una frase que pueda llegar a ser compleja por el rendimiento del mismo.

## Conclusión

Debido a los recursos limitados que posee el entorno de ejecucion el rendimiento general del modelo es bastante bajo llegando al punto de que las predicciones no forman una palabra coherente (lenguaje español). Esto puede deberse a la capa de embedding, debido a que vectoriza los datos en una dimension de 512 se necesita un mayor consumo de recursos, los cuales no se poseen. Cabe destacar que se realizara un estudio tomando un enfoque diferente donde se pueda realizar un preprocesamiento de datos utilizando formato one-hot-encoding, lo cual permitira llevarlo a un enfoque mas de multi clases con matrices de 0 y 1, esto hara que la capa de embedding no sea necesaria y el modelo no tenga que consumir tantos recursos para procesarlo dando un margen de recursos para poder distribuir de mejor forma y mejorar asi el rendimiento del mismo.

Enlace a investigacion con one-hot-encoding (Notebook 2):